In [0]:
## mount files:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os, os.path
folder = 'Moocs/coursera_Duke/notebooks'
os.chdir('/content/drive/My Drive/'+folder)

In [0]:
# 1. Reducing the number of intensity levels in an image from 256 to 2, in integer powers of 2
# 2. Replace the value of every pixel by the average of the values in its (num x num) neighborhood
# 3. Rotate the image by 45 and 90 degrees

import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
from skimage import color
from scipy import signal
from scipy import ndimage
import math

# required to display matplotlib plots in notebooks
%matplotlib inline

def read_image(img_path):
    return np.array(Image.open(img_path)).astype('float64')

def normalize_img(np_img): # range: 0 -> 1
    # the formula is: (x - min(x)) / (max(x) - min(x))
    if (np.max(np_img) == np.min(np_img)):
        return False
    return (np_img - np.min(np_img)) / (np.max(np_img) - np.min(np_img))

def show_img(np_img, is_grayscale):
    if is_grayscale:
        plt.imshow(np_img, cmap=plt.get_cmap('gray'))
    else:
        plt.imshow(np_img)
    plt.show()

def convert_img_colors(dest_color, np_img):
    if dest_color == 'RGB':
        return color.gray2rgb(np_img)
    elif dest_color == 'GS':
        return color.rgb2gray(np_img)
    else:
        return False

def calc_equal_bins(start, stop, num_of_bins):
    return np.linspace(start, stop, num=num_of_bins)

def make_histogram(one_channel_img, levels):
    if (len(one_channel_img.shape) > 2):
        return False
    hist_bins = calc_equal_bins(np.min(one_channel_img), 
                                np.max(one_channel_img), levels)
    hist, bin_edges = np.histogram(one_channel_img, bins=hist_bins)
    plt.hist(one_channel_img.ravel(), bins=hist_bins)
    plt.show()

def reduce_levels_intensity(desired_intensity_levels, img):
    # desired_intensity_levels should be 2^k for k in [1..8]
    if (desired_intensity_levels > 256 or not math.log(desired_intensity_levels, 2.0).is_integer()):
        return False
    # grayscale image
    if (len(img.shape) != 2):
        return False
    divisor = 256 / desired_intensity_levels
    return np.floor(img / divisor) * divisor

def create_average_filter(filter_size):
    if ((filter_size % 2) == 0):
        return False
    img_filter = np.repeat(np.array([1]), filter_size)
    img_filter = img_filter.reshape(1, img_filter.shape[0])
    img_filter = np.repeat(img_filter, filter_size, axis=0).astype('float64')
    img_filter /= np.sum(img_filter)
    return img_filter

def convolve_average_filter(img, filter_size):
    # grayscale image
    if (len(img.shape) != 2):
        return False
    img_filter = create_average_filter(filter_size)
    return signal.convolve2d(img, img_filter, boundary='symm', mode='same')

def rotate_image(img, angle):
    return ndimage.rotate(img, angle)

def main():
    main_path = './data'
    image_name = 'vietnam.jpg'
    desired_intensity_levels = 32
    img = read_image(os.path.join(main_path, image_name))
    img = convert_img_colors('GS', img)
    img = reduce_levels_intensity(desired_intensity_levels, img)
    img = normalize_img(img)
    filter_size = 3
    img = convolve_average_filter(img, filter_size)
    make_histogram(img, desired_intensity_levels)
    angle = 45
    img = rotate_image(img, angle)
    show_img(img, True)

main()